# Chained function calling with OpenAPI Spec

This notebook demonstrates a straightforward way to chain function calls together. We have created a fake OpenAPI spec for "froge" characters using `gpt-4`. We then transform the OpenAPI spec into a set of functions that can be supplied to the Chat completion API. The model intelligently chooses to output a JSON object containing arguments to call those functions. The Chat completions API does not call the function; instead, the model generates JSON you can use to call the function.

This notebook contains the following 2 sections. First, we convert a sample OpenAPI spec into a list of functions and arguments. See API Reference. Then we take that list of functions, a user instruction, and execute the functions sequentially.

In [ ]:
!pip install -q jsonref
!pip install -q openai


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import json
import jsonref
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

## Parsing OpenAPI spec and converting it into OpenAI functions

Let's parse a fake OpenAPI spec into function specifications for the OpenAI Chat completion API. 

In [ ]:
def parse_functions(openapi_spec):    

    functions = []

    # iterate through the paths and methods specified in the OpenAPI spec
    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            spec = jsonref.replace_refs(spec_with_ref)
            function_name = spec.get("operationId")
            
            # gather the description, request body and parameters from the spec
            desc = spec.get("description") or spec.get("summary", "")
            req_body = spec.get("requestBody", {}).get("content", {}).get("application/json", {}).get("schema")
            params = spec.get("parameters", [])
            
            schema = {"type": "object", "properties": {}}
            if req_body:
                schema["properties"]["requestBody"] = req_body
            
            if params:
                param_properties = {param["name"]: param["schema"] for param in params}
                schema["properties"]["parameters"] = {"type": "object", "properties": param_properties}
            
            functions.append({"name": function_name, "description": desc, "parameters": schema})

    return functions

url = "https://gist.githubusercontent.com/shyamal-anadkat/d44674a87778796222bdb8fa9158ad47/raw/030d173d53c55d806a93976705cf1c5f9e9c5240/frogeapi.json"
openapi_spec = jsonref.loads(requests.get(url).content)

functions = parse_functions(openapi_spec)

display(functions)

[{'name': 'listFroges',
  'description': 'List all froge characters',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'createFroge',
  'description': 'Create a new froge character',
  'parameters': {'type': 'object',
   'properties': {'requestBody': {'type': 'object',
     'properties': {'id': {'type': 'string'},
      'name': {'type': 'string'},
      'age': {'type': 'integer'}},
     'required': ['name', 'age']}}}},
 {'name': 'getFrogeById',
  'description': 'Retrieve a froge character by ID',
  'parameters': {'type': 'object',
   'properties': {'parameters': {'type': 'object',
     'properties': {'id': {'type': 'string'}}}}}},
 {'name': 'deleteFroge',
  'description': 'Delete a froge character by ID',
  'parameters': {'type': 'object',
   'properties': {'parameters': {'type': 'object',
     'properties': {'id': {'type': 'string'}}}}}},
 {'name': 'updateFrogeName',
  'description': "Update a froge character's name by ID",
  'parameters': {'type': 'object',
   'proper

## Chaining function calls

Let's pass the fake function specifications, along with a user instruction, and write simple logic for chaining those functions together sequentially. 

In [ ]:
FROGE_PROMPT = """
You are a helpful froge. Respond to the following prompt by using function_call.

Get all the froges. Then create a new 2-year old froge named dalle3, with a random numerical id. Then look for a froge named davinci. Then delete froge with id 2456.
"""
messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": FROGE_PROMPT}
]

# Maximum number of chained calls allowed to prevent infinite or lengthy loops
MAX_CHAINED_CALLS = 5

def get_openai_response(functions, messages):
    return openai.ChatCompletion.create(
        model='gpt-3.5-turbo-16k-0613',
        functions=functions,
        function_call="auto", # "auto" means the model can pick between generating a message or calling a function.
        temperature=0,
        messages=messages
    )

def process_chained_calls(functions, messages):
    stack = 0
    while stack < MAX_CHAINED_CALLS:
        response = get_openai_response(functions, messages)
        message = response["choices"][0]["message"]
        
        if message.get("function_call"):
            display(f"Function call number: {stack + 1}")
            display(message)
            messages.append(message)
            stack += 1
        else:
            display(message)
            break
    
    if stack >= MAX_CHAINED_CALLS:
        display(f"Reached max chained function calls: {MAX_CHAINED_CALLS}")

process_chained_calls(functions, messages)

'Function call number: 1'

<OpenAIObject at 0x7f1d2fa41040> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "listFroges",
    "arguments": "{}"
  }
}

'Function call number: 2'

<OpenAIObject at 0x7f1d2fb4f9f0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "createFroge",
    "arguments": "{\n  \"requestBody\": {\n    \"id\": \"1234\",\n    \"name\": \"dalle3\",\n    \"age\": 2\n  }\n}"
  }
}

'Function call number: 3'

<OpenAIObject at 0x7f1d2fae0d10> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "getFrogeByName",
    "arguments": "{\n  \"parameters\": {\n    \"name\": \"davinci\"\n  }\n}"
  }
}

'Function call number: 4'

<OpenAIObject at 0x7f1d2fa706d0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "deleteFroge",
    "arguments": "{\n  \"parameters\": {\n    \"id\": \"2456\"\n  }\n}"
  }
}

<OpenAIObject at 0x7f1d2fa70ae0> JSON: {
  "role": "assistant",
  "content": "Here are the steps you requested:\n\n1. Get all the froges.\n2. Create a new 2-year old froge named dalle3 with a random numerical id.\n3. Look for a froge named davinci.\n4. Delete the froge with id 2456.\n\nPlease note that I have executed the function calls, but I haven't received any specific responses from the server."
}

<img src="image-20231013-013652.png" width="" align="" />

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=baca1bec-71a4-4a27-a9d1-210d39960b44' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>